In [ ]:
import os
import pandas as pd
import pickle

In [ ]:
folderlist = ['asparagus', 'bambooshoots', 'betel', 'broccoli', 'cauliflower', 'chinesecabbage', 'chinesechives', 'custardapple', 'grape', 'greenhouse', 'greenonion', 'kale', 'lemon', 'lettuce', 'litchi', 'longan', 'loofah', 'mango', 'onion', 'others', 'papaya', 'passionfruit', 'pear', 'pennisetum', 'redbeans', 'roseapple', 'sesbania', 'soybeans', 'sunhemp', 'sweetpotato', 'taro', 'tea', 'waterbamboo']
class_list = sorted(folderlist)

In [ ]:
with open("train_emb_swinv2_base", "rb") as fp:
    train_emb_list_swinv2, train_label_list_swinv2, valid_emb_list_swinv2, valid_label_list_swinv2 = pickle.load(fp)
    
with open("train_emb_convnext_base", "rb") as fp:
    train_emb_list_convnext_base, train_label_list_convnext_base, valid_emb_list_convnext_base, valid_label_list_convnext_base = pickle.load(fp)
    
with open("train_emb_convnext_large", "rb") as fp:
    train_emb_list_convnext_large, train_label_list_convnext_large, valid_emb_list_convnext_large, valid_label_list_convnext_large = pickle.load(fp)
    
with open("train_emb_effnetv2_large", "rb") as fp:
    train_emb_list_effnetv2, train_label_list_effnetv2, valid_emb_list_effnetv2, valid_label_list_effnetv2 = pickle.load(fp)

In [ ]:
train_emb_list = np.hstack((train_emb_list_swinv2, train_emb_list_convnext_base, train_emb_list_convnext_large, train_emb_list_effnetv2))
valid_emb_list = np.hstack((valid_emb_list_swinv2, valid_emb_list_convnext_base, valid_emb_list_convnext_large, valid_emb_list_effnetv2))

In [ ]:
train_label_list = train_label_list_convnext_large
valid_label_list = valid_label_list_convnext_large

In [ ]:
with open("test_emb_swinv2_base", "rb") as fp:
    test_emb_list_swinv2, test_file_list_swinv2 = pickle.load(fp)
    
with open("test_emb_convnext_base", "rb") as fp:
    test_emb_list_convnext_base, test_file_list_convnext_base = pickle.load(fp)
    
with open("test_emb_convnext_large", "rb") as fp:
    test_emb_list_convnext_large, test_file_list_convnext_large = pickle.load(fp)
    
with open("test_emb_effnetv2_base", "rb") as fp:
    test_emb_list_effnetv2, test_file_list_effnetv2 = pickle.load(fp)

In [ ]:
test_emb_list = np.hstack((test_emb_list_swinv2, test_emb_list_convnext_base, test_emb_list_convnext_large, test_emb_list_effnetv2))

In [ ]:
test_file_list = test_file_list_convnext_large

# XGBoost

In [ ]:
from xgboost import XGBClassifier

xgbmodel = XGBClassifier(
    objective='multi:softmax',
    num_class=33,
    tree_method='gpu_hist',
    gpu_id='0',
    seed=1224,
    
    eta =0.1,
    n_estimators=4600,
    max_depth = 6,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
)
eval_set = [(valid_emb_list, valid_label_list), (train_emb_list, train_label_list)]

xgbmodel.fit(
    train_emb_list,
    train_label_list,
    eval_metric=['merror'],
    eval_set=eval_set,
    verbose=True,
)

# Test

In [ ]:
def test(model):
    test_pred_list = model.predict(test_emb_list)
    
    test_label_list = [class_list[index] for index in test_pred_list]
    
    data = {
        'filename': test_file_list,
        'label': test_label_list,
    }

    df = pd.DataFrame(data)

    df.to_csv('test.csv', index=False)

In [ ]:
test(xgbmodel)